# PURPOSE OF THE NOTEBOOK

This notebook is designed to format Xenium data to adata and, if needed, keep only nuclear information of reads a certain distance from the nuclei.

# Loading the packages

In [3]:
import os.path
from scipy.io import mmread
#import xb.formatting as xf
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import scipy as sp
import matplotlib.pyplot as plt
import gzip
import shutil
import os.path
from scipy.io import mmread
import tifffile as tf
from scipy.spatial import ConvexHull, convex_hull_plot_2d
from anndata import AnnData
import json
#from xb.formatting import format_background

# Example of formatting

We input the path wher our original xenium data is and we use ```format_xenium_data``` to transform the datasets

In [4]:
def format_xenium_adata_2023_parquet(path,tag,output_path):
    #decompress
    if os.path.isfile(path+'/transcripts.csv')==False:
        with gzip.open(path+'/transcripts.csv.gz', 'rb') as f_in:
            with open(path+'/transcripts.csv', 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

    if os.path.isfile(path+'/cell_feature_matrix/barcodes.tsv')==False:
        with gzip.open(path+'/cell_feature_matrix/barcodes.tsv.gz', 'rb') as f_in:
            with open(path+'/cell_feature_matrix/barcodes.tsv', 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

    if os.path.isfile(path+'/cell_feature_matrix/features.tsv')==False:
        with gzip.open(path+'/cell_feature_matrix/features.tsv.gz', 'rb') as f_in:
            with open(path+'/cell_feature_matrix/features.tsv', 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

    if os.path.isfile(path+'/cell_feature_matrix/matrix.mtx')==False:
        with gzip.open(path+'/cell_feature_matrix/matrix.mtx.gz', 'rb') as f_in:
            with open(path+'/cell_feature_matrix/matrix.mtx', 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

    if os.path.isfile(path+'/cells.csv')==False:
        with gzip.open(path+'/cells.csv.gz', 'rb') as f_in:
            with open(path+'/cells.csv', 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    a = mmread(path+'/cell_feature_matrix/matrix.mtx')
    ad=a.todense()
    cell_info=pd.read_csv(path+r"/cells.csv")
    features=pd.read_csv(path+'/cell_feature_matrix/features.tsv',sep='\t',header=None,index_col=0)
    barcodes=pd.read_csv(path+'/cell_feature_matrix/barcodes.tsv',header=None,index_col=0)
    adata=sc.AnnData(ad.transpose(),obs=cell_info,var=features)
    adata.var.index.name='index'
    adata.var.columns=['gene_id','reason_of_inclusion']
    import json
    # Opening JSON file
    f = open(path+'/gene_panel.json')
    # returns JSON object as 
    # a dictionary
    data = json.load(f)
    # Iterating through the json
    # list
    geness=[]
    idss=[]
    descriptorss=[]
    for r in range(len(data['payload']['targets'])):
        geness.append(data['payload']['targets'][r]['type']['data']['name'])
        try:
            idss.append(data['payload']['targets'][r]['type']['data']['id'])
        except:
            idss.append('newid_'+str(r)) 
        try:
            descriptorss.append(data['payload']['targets'][r]['type']['descriptor'])
        except:
            descriptorss.append('other')
    # Closing file
    f.close()

    dict_inpanel=dict(zip(geness,descriptorss))
    dict_ENSEMBL=dict(zip(geness,idss))
    adata.var['Ensembl ID']=adata.var['gene_id'].map(dict_ENSEMBL)
    adata.var['in_panel']=adata.var['gene_id'].map(dict_inpanel)
    transcripts=pd.read_parquet(path+'/transcripts.parquet')
    adata.uns['spots']=transcripts
    try:
        UMAP=pd.read_csv(path+'/analysis/umap/gene_expression_2_components/projection.csv',index_col=0)
        adata.obsm['X_umap']=np.array(UMAP)
        TSNE=pd.read_csv(path+'/analysis/tsne/gene_expression_2_components/projection.csv',index_col=0)
        adata.obsm['X_tsne']=np.array(TSNE)
        PCA=pd.read_csv(path+'/analysis/PCA/gene_expression_10_components/projection.csv',index_col=0)
        adata.obsm['X_pca']=np.array(PCA)
        clusters=pd.read_csv(path+'/analysis/clustering/gene_expression_graphclust/clusters.csv',index_col=0)
        adata.obs['graph_clusters']=list(clusters['Cluster'].astype(str))
        kmeans2=pd.read_csv(path+'/analysis/clustering/gene_expression_kmeans_2_clusters/clusters.csv',index_col=0)
        adata.obs['kmeans2_clusters']=list(kmeans2['Cluster'].astype(str))
        kmeans3=pd.read_csv(path+'/analysis/clustering/gene_expression_kmeans_2_clusters/clusters.csv',index_col=0)
        adata.obs['kmeans3_clusters']=list(kmeans3['Cluster'].astype(str))
        kmeans4=pd.read_csv(path+'/analysis/clustering/gene_expression_kmeans_2_clusters/clusters.csv',index_col=0)
        adata.obs['kmeans4_clusters']=list(kmeans4['Cluster'].astype(str))
        kmeans5=pd.read_csv(path+'/analysis/clustering/gene_expression_kmeans_2_clusters/clusters.csv',index_col=0)
        adata.obs['kmeans5_clusters']=list(kmeans5['Cluster'].astype(str))
        kmeans6=pd.read_csv(path+'/analysis/clustering/gene_expression_kmeans_2_clusters/clusters.csv',index_col=0)
        adata.obs['kmeans6_clusters']=list(kmeans6['Cluster'].astype(str))
        kmeans7=pd.read_csv(path+'/analysis/clustering/gene_expression_kmeans_2_clusters/clusters.csv',index_col=0)
        adata.obs['kmeans7_clusters']=list(kmeans7['Cluster'].astype(str))
        kmeans8=pd.read_csv(path+'/analysis/clustering/gene_expression_kmeans_2_clusters/clusters.csv',index_col=0)
        adata.obs['kmeans8_clusters']=list(kmeans8['Cluster'].astype(str))
        kmeans9=pd.read_csv(path+'/analysis/clustering/gene_expression_kmeans_2_clusters/clusters.csv',index_col=0)
        adata.obs['kmeans9_clusters']=list(kmeans9['Cluster'].astype(str))
        kmeans10=pd.read_csv(path+'/analysis/clustering/gene_expression_kmeans_2_clusters/clusters.csv',index_col=0)
        adata.obs['kmeans10_clusters']=list(kmeans10['Cluster'].astype(str))
    except:
        print('UMAP and clusters_could not be recovered')
    adata.X=sp.sparse.csr_matrix(adata.X)
    adata.uns['spots']=adata.uns['spots'].fillna(0)
    adata.uns['spots']['fov_name']=adata.uns['spots']['fov_name'].astype(str)
    adata.write(output_path+tag+'.h5ad')
    return adata

def keep_nuclei_and_quality(adata1,distance_to_nucleus=1,qvmin=20):

    subset1=adata1.uns['spots'].loc[adata1.uns['spots']['nucleus_distance']<distance_to_nucleus,:]
    subset1=subset1[subset1['qv']>qvmin]
    ct1=pd.crosstab(subset1['cell_id'],subset1['feature_name'])
    adataobs=adata1.obs.loc[adata1.obs['cell_id'].isin(ct1.index),:]
    av=adata1.var.index[adata1.var.index.isin(ct1.columns)]#.isin(adata1.var.index)
    ct1=ct1.loc[:,av]
    adataobs.index=adataobs['cell_id']
    adataobs.index.name='ind'
    ct1=ct1.loc[ct1.index.isin(adataobs['cell_id']),:]
    adata1nuc=sc.AnnData(np.array(ct1),obs=adataobs)#,var=adata1.var)
    adata1nuc.var.index=ct1.columns
    return adata1nuc

# Format the data

First we define the path where our raw data is, in ´genpath´. We also define where to save all the other outputs

In [8]:
genpath='/media/sergio/Meninges/CRC/Xenium/'
output_path=r'/media/sergio/Meninges/CRC/xenium_processing/unprocessed_adata/' # path where to save the normal adata
nuclei_path=r'/media/sergio/Meninges/CRC/xenium_processing/nuclei_adata/' # path where to save the nuclei adata
combined_path=r'/media/sergio/Meninges/CRC/xenium_processing/combined_adata/' # path where to save combined all filtered AnnData objects


Now we format each dataset in the path defined in genpath. This might take a while

In [9]:
fls=os.listdir(genpath)
fls_filt=[fi for fi in fls if not '.zip' in fi]
for f in fls_filt:
    print(f)
    path=genpath+f
    tag=f.split('__')[1]+'_'+f.split('__')[2]
    ad=format_xenium_adata_2023_parquet(path,tag,output_path)
    format_background(path)

output-XETG00213__0021489__Region_1__20240125__150400


/home/sergio/anaconda3/envs/xb/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
/home/sergio/anaconda3/envs/xb/lib/python3.7/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


UMAP and clusters_could not be recovered
output-XETG00213__0021489__Region_2__20240125__150400
UMAP and clusters_could not be recovered
output-XETG00213__0021492__Region_1__20240125__150400
UMAP and clusters_could not be recovered
output-XETG00213__0021492__Region_2__20240125__150400
UMAP and clusters_could not be recovered


# Keep only nuclear genes

Now we read all the files we created in the previous step and extract only the reads with a minimum quality of 20 and situated in the nuclei (with a nuclei distance <0.1). In xenium's data, all reads at a distance to nucleus of 0 are, indeed, in the nuclei

In [11]:
files=os.listdir(output_path)
allf=[]
for f in files[:]:
    print(f)
    adata1=sc.read(output_path+f)
    adata1.var.index=adata1.var['gene_id']
    adata_f1=keep_nuclei_and_quality(adata1,distance_to_nucleus=0.1,qvmin=20)
    adata_f1.obs['sample']=f.split('.')[0]
    adata_f1.write(nuclei_path+f)
    allf.append(adata_f1)

0021489_Region_1.h5ad


/home/sergio/anaconda3/envs/xb/lib/python3.7/site-packages/ipykernel_launcher.py:111: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.


0021489_Region_2.h5ad
0021492_Region_1.h5ad
0021492_Region_2.h5ad


Finally we concatenate all the datasets into one adata object, saved in the combined path 

In [13]:
adata=sc.concat(allf)
adata.write(combined_path+'adata_input.h5ad')

/home/sergio/anaconda3/envs/xb/lib/python3.7/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
